In [68]:
import pandas as pd
import pickle
from glob import glob

Columns to keep
- National Provider Identifier -- 1000 non-null int64
- Last Name/Organization Name of the Provider -- 1000 non-null object
- Entity Type of the Provider -- 1000 non-null object
- City of the Provider -- 1000 non-null object
- Zip Code of the Provider -- 1000 non-null int64
- State Code of the Provider -- 1000 non-null object
- Provider Type -- 1000 non-null object
- Place of Service -- 1000 non-null object
- HCPCS Code -- 1000 non-null object
- HCPCS Description -- 1000 non-null object
- Number of Services -- 1000 non-null float64
- Number of Medicare Beneficiaries -- 1000 non-null int64
- Number of Distinct Medicare Beneficiary/Per Day Services -- 1000 non-null int64
- Average Medicare Allowed Amount -- 1000 non-null float64
- ADD:  Year (in each df on import)

In [69]:
%%time
#option 1
cols = ['National Provider Identifier',
'Last Name/Organization Name of the Provider',
'Entity Type of the Provider', 
'City of the Provider',
'Zip Code of the Provider', 
'State Code of the Provider',
'Provider Type',
'Place of Service',
'HCPCS Code',
'HCPCS Description',
'Number of Services',
'Number of Medicare Beneficiaries',
'Number of Distinct Medicare Beneficiary/Per Day Services',
'Average Medicare Allowed Amount']

payment_rows = []
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv',
                         usecols = cols,
                         chunksize = 1000):
        chunk['year'] = 2017
        payment_rows.append(chunk)
        
df_payments_2017 = pd.concat(payment_rows, ignore_index=True)

CPU times: user 1min 25s, sys: 25.3 s, total: 1min 51s
Wall time: 1min 54s


In [70]:
df_payments_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9847443 entries, 0 to 9847442
Data columns (total 15 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   Entity Type of the Provider                               object 
 3   City of the Provider                                      object 
 4   Zip Code of the Provider                                  object 
 5   State Code of the Provider                                object 
 6   Provider Type                                             object 
 7   Place of Service                                          object 
 8   HCPCS Code                                                object 
 9   HCPCS Description                                         object 
 10  Number of Services            

In [71]:
%%time
#option 2

payment_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         chunksize = 1000, usecols = cols):
    payment_rows.append(chunk)
    
df_payments_2017 = pd.concat(payment_rows, ignore_index=True)
df_payments_2017['year'] = 2017

CPU times: user 1min 15s, sys: 22.5 s, total: 1min 38s
Wall time: 1min 38s


In [72]:
%%time
#option 3 without chunks and without for loop

df_payments_2017 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                               usecols = cols, low_memory = False)
df_payments_2017['year'] = 2017

CPU times: user 31.3 s, sys: 13 s, total: 44.3 s
Wall time: 49.1 s


In [73]:
print(df_payments_2017.shape)
df_payments_2017.head()

(9847443, 15)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Provider Type,Place of Service,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,100.0,96,100,73.398800,2017
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,25,26,100.080000,2017
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,51,52,136.380000,2017
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59,59,190.363729,2017
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16,16,101.680000,2017


In [74]:
#make all columns lowercase & replace spaces with _ and / with _
df_payments_2017.columns = map(str.lower,df_payments_2017.columns)
df_payments_2017.columns = df_payments_2017.columns.str.replace(" ", "_").str.replace("/", "_")

df_payments_2017.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,100.0,96,100,73.398800,2017
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,25,26,100.080000,2017
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,51,52,136.380000,2017
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59,59,190.363729,2017
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16,16,101.680000,2017


In [75]:
df_payments_2017.to_pickle('../data/df_payments_2017.pkl')

#### Option 3 is the fastest to read in data

In [76]:
%%time
#read in year 2016

df_payments_2016 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016.csv', 
                               usecols = cols)
df_payments_2016['year'] = 2016

<string>:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 30 s, sys: 3.93 s, total: 33.9 s
Wall time: 34.6 s


In [77]:
print(df_payments_2016.shape)
df_payments_2016.head()

(9714896, 15)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Provider Type,Place of Service,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,57.0,55,57,72.743158,2016
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,38.0,38,38,135.010000,2016
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,23.0,23,23,189.239565,2016
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",20.0,20,20,100.750000,2016
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",96.0,87,96,136.250000,2016


In [78]:
df_payments_2016.columns = map(str.lower,df_payments_2016.columns)
df_payments_2016.columns = df_payments_2016.columns.str.replace(" ", "_").str.replace("/", "_")

df_payments_2016.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,57.0,55,57,72.743158,2016
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,38.0,38,38,135.010000,2016
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,23.0,23,23,189.239565,2016
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",20.0,20,20,100.750000,2016
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",96.0,87,96,136.250000,2016


In [79]:
df_payments_2016.to_pickle('../data/df_payments_2016.pkl')

In [80]:
%%time
#read in year 2015

df_payments_2015 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2015.csv', 
                               usecols = cols)
df_payments_2015['year'] = 2015

<string>:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 30.5 s, sys: 5.7 s, total: 36.2 s
Wall time: 38.1 s


In [81]:
print(df_payments_2015.shape)
df_payments_2015.head()

(9497892, 15)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Provider Type,Place of Service,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",220.0,215.0,220.0,205.185955,2015


In [82]:
df_payments_2015.columns = map(str.lower,df_payments_2015.columns)
df_payments_2015.columns = df_payments_2015.columns.str.replace(" ", "_").str.replace("/", "_")

df_payments_2015.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",220.0,215.0,220.0,205.185955,2015


Drop blank row in 2015 file - idenified this in data.cms.gov file

In [83]:
df_payments_2015[df_payments_2015.national_provider_identifier == 1]

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
7205022,1,CPT copyright 2014 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015


In [84]:
df_payments_2015 = df_payments_2015.drop(labels = 7505022)

In [85]:
df_payments_2015.to_pickle('../data/df_payments_2015.pkl')

Combine all years files and pickle

In [91]:
%%time
#concat all years 2015-2017
df_payments_combined = pd.concat([df_payments_2015, df_payments_2016, df_payments_2017], ignore_index= True)
#other way when we globbed
#df_payments_combined = pd.concat((pd.read_pickle(file)
 #              for file in list_payments_combined), ignore_index=True)

CPU times: user 4.02 s, sys: 11 s, total: 15 s
Wall time: 20 s


In [92]:
print(df_payments_combined.shape)
df_payments_combined.head()

(29060230, 15)


,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",220.0,215.0,220.0,205.185955,2015


In [93]:
df_payments_combined.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29060230 entries, 0 to 29060229
Data columns (total 15 columns):
 #   Column                                                    Non-Null Count     Dtype  
---  ------                                                    --------------     -----  
 0   national_provider_identifier                              29060230 non-null  int64  
 1   last_name_organization_name_of_the_provider               29059803 non-null  object 
 2   entity_type_of_the_provider                               29060229 non-null  object 
 3   city_of_the_provider                                      29060224 non-null  object 
 4   zip_code_of_the_provider                                  29060225 non-null  object 
 5   state_code_of_the_provider                                29060229 non-null  object 
 6   provider_type                                             29060229 non-null  object 
 7   place_of_service                                          29060229 non

Save combined file to a pickle file

In [94]:
df_payments_combined.to_pickle('../data/df_payments_combined.pkl')